In [ ]:
import numpy as np
import pandas as pd
from random import choices, seed
import matplotlib.pyplot as plt
import sys
import scipy
import scipy.stats
import random

In [155]:
clusterPath = '/Users/jones/Downloads/resultsClusterSizesALLAllAges.csv'

In [156]:
dfAll = pd.read_csv(clusterPath)

In [158]:
def twoNucInCell(row):
    return np.min([row['ki67'], row['clusterSize']])

In [159]:
dfAll['ki67Adjust'] = dfAll.apply(lambda row: twoNucInCell(row), axis = 1)

In [160]:
def KL(a, b):
    a = np.asarray(a, dtype=np.float)
    b = np.asarray(b, dtype=np.float)

    return np.sum(np.where(a != 0, a * np.log(a / b), 0))

Simulation

In [ ]:
age = "1-21"
df = dfAll.loc[(dfAll.age == age)]

In [189]:
dfActive = df.groupby('clusterSize').sum()
dfActive = dfActive.reset_index(drop=False)
dfActive['percentage'] = dfActive['ki67Adjust']/np.sum(dfActive['ki67Adjust'])
for index, row in dfActive.iterrows():
    if index == 0:
        activeCellClustersReal = np.full(int(row['ki67Adjust']), int(row['clusterSize']))
    else:
        activeCellClustersReal = np.concatenate((activeCellClustersReal, np.full(int(row['ki67Adjust']), int(row['clusterSize']))), axis=0)

In [192]:
# Assume `df.clusterSize` contains your initial cluster sizes
unique, counts = np.unique(df.clusterSize, return_counts=True)

In [ ]:
#cluster needs to appear more than 10times; if you want to add this filter

mask = counts >= 10
unique = unique[mask]
counts = counts[mask]

In [ ]:
# for exponential weights; fill in zeros for empty clusters

# 0-7: 9 insert
#1-21: 14,15,16,18
unique = np.insert(unique, 13, 14)
counts = np.insert(counts, 13, 0)

unique = np.insert(unique, 14, 15)
counts = np.insert(counts, 14, 0)

unique = np.insert(unique, 15, 16)
counts = np.insert(counts, 15, 0)

unique = np.insert(unique, 17, 18)
counts = np.insert(counts, 17, 0)



In [ ]:
seed(1234)  # Set seed for reproducibility

maximumPerCluster = counts * unique

# Exponential weights based on cluster size
weights = np.exp(unique - 1)  # Exponential function prioritizing larger clusters
weights /= weights.sum()  # Normalize weights to form a probability distribution

# Placeholder to store results of each simulation
resultsExp = []

# Run simulations
for i in range(1000):
    # Initialize for each simulation
    simulation_result = []
    max_per_cluster = maximumPerCluster.copy()  # Reset max capacities for each simulation
    current_unique = unique.copy()  # Copy unique array for dynamic adjustment
    current_weights = weights.copy()  # Copy weights array for dynamic adjustment
    
    # Sample each proliferative cell individually
    for _ in range(np.sum(df.ki67Adjust)):
        # Choose a cluster based on current weights
        selected_cluster = choices(current_unique, current_weights)[0]
        index = np.where(current_unique == selected_cluster)[0][0]  # Find index in current arrays

        # Add the chosen cluster to the result
        simulation_result.append(selected_cluster)
        max_per_cluster[index] -= 1  # Decrease capacity for this cluster

        # Remove the cluster if its capacity is 0
        if max_per_cluster[index] == 0:
            current_unique = np.delete(current_unique, index)
            current_weights = np.delete(current_weights, index)
            max_per_cluster = np.delete(max_per_cluster, index)

            # Re-normalize weights
            current_weights /= current_weights.sum()

    resultsExp.append(simulation_result)

In [ ]:
#exponential function
seedNumber = 1234*1
random.seed(seedNumber)
distanceRealWasserstein = []
negControlWasserstein = []
distanceJensenShannon = []
negControlJensenShannon = []
distanceKL = []
negControlKL = []


for i in range(1000):
    distanceRealWasserstein.append(scipy.stats.wasserstein_distance(activeCellClustersReal, resultsExp[i]))

    uniqueSample, countsSample = np.unique(resultsExp[i], return_counts=True)
    percentageSimuli = []
    for index, row  in dfActive.iterrows():
        if len(countsSample[np.where(uniqueSample==row['clusterSize'])]) == 0:
            percentageSimuli.append(sys.float_info.epsilon)
        else:
            percentageSimuli.append(countsSample[np.where(uniqueSample==row['clusterSize'])][0]/np.sum(df.ki67Adjust))
    percentageSimuli = np.asarray(percentageSimuli)
    distanceJensenShannon.append(scipy.spatial.distance.jensenshannon(dfActive['percentage'].values, percentageSimuli))
    distanceKL.append(KL(dfActive['percentage'].values,percentageSimuli))
    for j in range(i+1,1000):
        negControlWasserstein.append(scipy.stats.wasserstein_distance(resultsExp[j], resultsExp[i]))
        uniqueSample, countsSample = np.unique(resultsExp[j], return_counts=True)
        percentageSimulj = []
        for index, row  in dfActive.iterrows():
            if len(countsSample[np.where(uniqueSample==row['clusterSize'])]) == 0:
                percentageSimulj.append(sys.float_info.epsilon)
            else:
                percentageSimulj.append(countsSample[np.where(uniqueSample==row['clusterSize'])][0]/np.sum(df.ki67Adjust))
        percentageSimulj = np.asarray(percentageSimulj)
        negControlJensenShannon.append(scipy.spatial.distance.jensenshannon(percentageSimuli, percentageSimulj))
        negControlKL.append(KL(percentageSimuli, percentageSimulj))

In [202]:
df_realDist = pd.DataFrame({'age': age, 'simulation': 'ExpData', 'Wasserstein': distanceRealWasserstein, 'JensenShannon': distanceJensenShannon, 'KL': distanceKL})
df_negControl= pd.DataFrame({'age': age, 'simulation': 'NegControl', 'Wasserstein': negControlWasserstein, 'JensenShannon': negControlJensenShannon, 'KL': negControlKL})

In [203]:
df_realDist.to_csv("/Users/jones/Downloads/realDistance_ExponentialData_1-21.csv", index = False)
df_negControl.to_csv("/Users/jones/Downloads/negativeControl_ExponentialData_1-21.csv", index = False)

In [ ]:
#fill in zeros for empty clusters
# 0-7: 9 insert
#1-21: 14,15,16,18

unique, countsCl = np.unique(df.clusterSize, return_counts=True)
unique = np.insert(unique, 13, 14)
countsCl = np.insert(countsCl, 13, 0)

unique = np.insert(unique, 14, 15)
countsCl = np.insert(countsCl, 14, 0)

unique = np.insert(unique, 15, 16)
countsCl = np.insert(countsCl, 15, 0)

unique = np.insert(unique, 17, 18)
countsCl = np.insert(countsCl, 17, 0)


In [231]:
#Markovian divison
seedNumber = 1234*1
random.seed(seedNumber)
distanceRealWasserstein = []
negControlWasserstein = []
distanceJensenShannon = []
negControlJensenShannon = []
distanceKL = []
negControlKL = []
dataCluster = []


counts = countsCl * unique
length = np.sum(counts)
weights = counts/length

for i in range(1000):
    dataCluster.append(choices(unique,weights, k= np.sum(df.ki67Adjust)))


for i in range(1000):
    distanceRealWasserstein.append(scipy.stats.wasserstein_distance(activeCellClustersReal, dataCluster[i]))

    uniqueSample, countsSample = np.unique(dataCluster[i], return_counts=True)
    percentageSimuli = []
    for index, row  in dfActive.iterrows():
        if len(countsSample[np.where(uniqueSample==row['clusterSize'])]) == 0:
            percentageSimuli.append(sys.float_info.epsilon)
        else:
            percentageSimuli.append(countsSample[np.where(uniqueSample==row['clusterSize'])][0]/np.sum(df.ki67Adjust))
    percentageSimuli = np.asarray(percentageSimuli)
    distanceJensenShannon.append(scipy.spatial.distance.jensenshannon(dfActive['percentage'].values, percentageSimuli))
    distanceKL.append(KL(dfActive['percentage'].values,percentageSimuli))
    for j in range(i+1,1000):
        negControlWasserstein.append(scipy.stats.wasserstein_distance(dataCluster[j], dataCluster[i]))
        uniqueSample, countsSample = np.unique(dataCluster[j], return_counts=True)
        percentageSimulj = []
        for index, row  in dfActive.iterrows():
            if len(countsSample[np.where(uniqueSample==row['clusterSize'])]) == 0:
                percentageSimulj.append(sys.float_info.epsilon)
            else:
                percentageSimulj.append(countsSample[np.where(uniqueSample==row['clusterSize'])][0]/np.sum(df.ki67Adjust))
        percentageSimulj = np.asarray(percentageSimulj)
        negControlJensenShannon.append(scipy.spatial.distance.jensenshannon(percentageSimuli, percentageSimulj))
        negControlKL.append(KL(percentageSimuli, percentageSimulj))



/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_43691/4152628166.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  a = np.asarray(a, dtype=np.float)
/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_43691/4152628166.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  b = np.asarray(b, dtype=np.float)
/var/folders/6r/

In [ ]:
df_realDist = pd.DataFrame({'age': age, 'simulation': 'ExpData', 'Wasserstein': distanceRealWasserstein, 'JensenShannon': distanceJensenShannon, 'KL': distanceKL})
df_negControl= pd.DataFrame({'age': age, 'simulation': 'NegControl', 'Wasserstein': negControlWasserstein, 'JensenShannon': negControlJensenShannon, 'KL': negControlKL})

In [233]:
df_realDist.to_csv('/Users/jones/Downloads/realDistance_MarkDiv_1-21.csv', index=False)
df_negControl.to_csv('/Users/jones/Downloads/negativeControl_MarkDiv_1-21.csv', index=False)

In [ ]:
# if you want to filter clusters, which appear more than 10 times
unique, counts = np.unique(df.clusterSize, return_counts=True)
mask = counts >= 10
unique = unique[mask]
counts = counts[mask]

In [ ]:
#fill in zeros for empty clusters
# 0-7: 9 insert
#1-21: 14,15,16,18

unique, counts = np.unique(df.clusterSize, return_counts=True)

unique = np.insert(unique, 13, 14)
counts = np.insert(counts, 13, 0)

unique = np.insert(unique, 14, 15)
counts = np.insert(counts, 14, 0)

unique = np.insert(unique, 15, 16)
counts = np.insert(counts, 15, 0)

unique = np.insert(unique, 17, 18)
counts = np.insert(counts, 17, 0)

In [ ]:
#Markovian cluster
seedNumber = 1234*2
random.seed(seedNumber)
distanceRealWasserstein = []
negControlWasserstein = []
distanceJensenShannon = []
negControlJensenShannon = []
distanceKL = []
negControlKL = []
dataCluster = []

#unique, counts = np.unique(df.clusterSize, return_counts=True)
weights = counts/np.sum(counts)
dataCluster = []
for i in range(1000):
    dataCluster.append(choices(unique,weights, k= np.sum(df.ki67Adjust)))

for i in range(1000):
    distanceRealWasserstein.append(scipy.stats.wasserstein_distance(activeCellClustersReal, dataCluster[i]))

    uniqueSample, countsSample = np.unique(dataCluster[i], return_counts=True)
    percentageSimuli = []
    for index, row  in dfActive.iterrows():
        if len(countsSample[np.where(uniqueSample==row['clusterSize'])]) == 0:
            percentageSimuli.append(sys.float_info.epsilon)
        else:
            percentageSimuli.append(countsSample[np.where(uniqueSample==row['clusterSize'])][0]/np.sum(df.ki67Adjust))
    percentageSimuli = np.asarray(percentageSimuli)
    distanceJensenShannon.append(scipy.spatial.distance.jensenshannon(dfActive['percentage'].values, percentageSimuli))
    distanceKL.append(KL(dfActive['percentage'].values,percentageSimuli))
    for j in range(i+1,1000):
        negControlWasserstein.append(scipy.stats.wasserstein_distance(dataCluster[j], dataCluster[i]))
        uniqueSample, countsSample = np.unique(dataCluster[j], return_counts=True)
        percentageSimulj = []
        for index, row  in dfActive.iterrows():
            if len(countsSample[np.where(uniqueSample==row['clusterSize'])]) == 0:
                percentageSimulj.append(sys.float_info.epsilon)
            else:
                percentageSimulj.append(countsSample[np.where(uniqueSample==row['clusterSize'])][0]/np.sum(df.ki67Adjust))
        percentageSimulj = np.asarray(percentageSimulj)
        negControlJensenShannon.append(scipy.spatial.distance.jensenshannon(percentageSimuli, percentageSimulj))
        negControlKL.append(KL(percentageSimuli, percentageSimulj))

In [ ]:
df_realDist = pd.DataFrame({'age': age, 'simulation': 'ExpData', 'Wasserstein': distanceRealWasserstein, 'JensenShannon': distanceJensenShannon, 'KL': distanceKL})
df_negControl= pd.DataFrame({'age': age, 'simulation': 'NegControl', 'Wasserstein': negControlWasserstein, 'JensenShannon': negControlJensenShannon, 'KL': negControlKL})

In [239]:
df_realDist.to_csv('/Users/jones/Downloads/realDistance_MarkCl_1-21.csv', index=False)
df_negControl.to_csv('/Users/jones/Downloads/negativeControl_MarkCl_1-21.csv', index=False)